In [ ]:
# @title Default title text
URL = "https://results.kongu.edu/mmxxivre/mmxxivreo.php" # @param {type:"string"}
student_database_path = "/content/AI-DS-DOB (1).xlsx" # @param {type:"string"}

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv


df = pd.read_excel(student_database_path)

data = df.to_dict(orient='records')

result_to_excel = []
columnes_names = ['roll_number']

with requests.Session() as session:
    for entry in data:
        roll = entry['Roll No.']
        dob = entry['Date of Birth']

        payload = {
            "regno": roll,
            "dob": dob
        }

        try:
            response = session.post(URL, data=payload)

            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html5lib')

            new = soup.find('div')
            if not new or not new.table:
                continue

            k = new.table.tbody.findAll('td')
            table = soup.findAll('table')[1]

            quote = {'roll_number': k[2].font.text}

            for row in table.findAll('tr')[1:]:
                subject_names = row.findAll('td')[2].text
                grade = row.findAll('th')[1].text
                quote[subject_names] = grade

                if subject_names not in columnes_names:
                    columnes_names.append(subject_names)

            result_to_excel.append(quote)

        except Exception as e:
            print(f"Error processing roll number {roll}: {e}")


filename = 'results.csv'
with open(filename, 'w', newline='') as f:
    w = csv.DictWriter(f, columnes_names)
    w.writeheader()
    w.writerows(result_to_excel)
